# Custom Chatbot Project

## Scenario
Because of the llm training date information regarding an actual bmw is difficult to get from an llm and sometimes it hallucinating some information. 
instead of connecting to an internal database which i can't use for this project, i decided to scrap the us bmw website to get some actual information
for 3 cars (ix xdrive 50, i7 M70 Sedan and the i5 M60 series) and feed this information into a prompt. One key aspect is that the information and answers are from and for the us market 


## Data collection

In [338]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import openai
import os
openai.api_key = os.getenv('OPENAI_API_KEY')
#DataPath
path = "./data/"


In [252]:
# Function to fetch a page
def fetch_page(url: str):
    headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    r = requests.get(url)
    if r.status_code == 200:
        return r.text
    else:
        print(r.status_code)
        return r.text

In [540]:
#research_url = https://www.bmwusa.com/vehicles/all-electric/i5/sedan/electric-bmw-m.html
#research_url2 = https://www.bmwusa.com/vehicles/all-electric/ix/suv/electric-bmw-m.html
#research_url3 =https://www.bmwusa.com/vehicles/all-electric/i7/sedan/electric-bmw-m.html

# Get the source of the page
#source = fetch_page(research_url)
# Write the source to disk
#with open("2025BMWiXxDrive50All-ElectricSUV_ Features&Specs.html", "w") as f:
#    f.write(source)
# Get the source of the page
#source = fetch_page(research_url2)
# Write the source to disk
#with open("BMWi7M70All-Electric LuxurySedan_BMWUSA.html", "w") as f:
#    f.write(source)
#source = fetch_page(research_url3)
# Write the source to disk
#with open("BMWi5M60All-ElectricSedan_BMWUSA.html", "w") as f:
#    f.write(source)
#had to store the pages localy because of access issues
    
# Load the source from disk
with open(path + "2025BMWiXxDrive50All-ElectricSUV_ Features&Specs.html", "r") as f:
    source = f.read()
with open(path + "BMWi7M70All-Electric LuxurySedan_BMWUSA.html", "r") as f:
    source2 = f.read()
with open(path + "BMWi5M60All-ElectricSedan_BMWUSA.html", "r") as f:
    source3 = f.read()

In [463]:
def parsePage2(source : str, vehicle_type : str) -> list: #As far as i understand the Text is placed
    soup = BeautifulSoup(source, 'html.parser')
    
    #e.g. Performance and efficieny
    headers = soup.find_all('div', { 'class' : "category-container--section-wrapper"})
    #print(len(headers))
    featureList : list = []
    for header in headers:
        #Idetify the headline
        features_Headline = header.find('h4', { 'class' : "section-header__copy h-left headline-6--bold"})
        if(features_Headline != None):
            #print(f'Headline : {features_Headline.text}')
            text : str = features_Headline.text
            #removing leading and  trainling whitespaces
            text = vehicle_type + " " + text.strip() + ': '
        #identify the features
        features = header.find_all('div', { 'class' : "body-text__copy body-copy text-content content-2 width-l"})
        for feature in features:
            #print(f'\t\t{ feature.text}')
            text += feature.text.strip()
        featureList.append(text)
    return featureList
featureList = parsePage2(source, "iX xDrive50")

In [464]:
len(featureList)

17

In [465]:
import pandas as pd
df_1 = pd.DataFrame(featureList, columns=['text'])
df_1

,text
0,iX xDrive50 Performance and Efficiency: Dual a...
1,"iX xDrive50 Handling, Ride and Braking: xDrive..."
2,iX xDrive50 Exterior: Aluminum Spaceframe with...
3,iX xDrive50 Interior Trim: Perforated Sensatec...
4,iX xDrive50 Connectivity: Connected Package Pr...
5,iX xDrive50 Audio System: SiriusXM® with 360L ...
6,iX xDrive50 Instrumentation and Controls: Mult...
7,iX xDrive50 Comfort and Convenience: Parking A...
8,iX xDrive50 Safety and Security: Mobility kitA...
9,iX xDrive50 Warranty: BMW Ultimate Care (for c...


In [467]:
featureList = parsePage2(source2, "i7 M70 Sedan")
df_test = pd.DataFrame(featureList, columns=['text'])
df_test


,text
0,i7 M70 Sedan Performance and Efficiency: Dual ...
1,"i7 M70 Sedan Handling, Ride and Braking: 4-whe..."
2,"i7 M70 Sedan Exterior: 21"" M Aerodynamic bi-co..."
3,i7 M70 Sedan Interior Trim: Extended Merino Le...
4,i7 M70 Sedan Connectivity: Remote Software Upg...
5,i7 M70 Sedan Audio System: Bowers and Wilkins ...
6,i7 M70 Sedan Instrumentation and Controls: Dyn...
7,i7 M70 Sedan Comfort and Convenience: Engine S...
8,i7 M70 Sedan Safety and Security: Adaptive Ful...
9,i7 M70 Sedan Warranty: BMW Ultimate Care (for ...


In [468]:
featureList = parsePage2(source2, "i7 M70 Sedan")
df_2 = pd.DataFrame(featureList, columns=['text'])

#df_all = pd.concat([df, pd.DataFrame(featureList, columns=['text'])], ignore_index=True)
featureList = parsePage2(source3, "i5 M60 All-Electric Sedan")
df_3 = pd.DataFrame(featureList, columns=['text'])
#df_all = pd.concat([df_all, pd.DataFrame(featureList, columns=['text'])], ignore_index=True)
df_all = pd.concat([df_1, df_2, df_3])

df_all = df_all.reset_index(drop=True)
df_all.to_csv(path + 'bmw_webcontent.csv', index=True, index_label='index')


In [469]:

df_all

,text
0,iX xDrive50 Performance and Efficiency: Dual a...
1,"iX xDrive50 Handling, Ride and Braking: xDrive..."
2,iX xDrive50 Exterior: Aluminum Spaceframe with...
3,iX xDrive50 Interior Trim: Perforated Sensatec...
4,iX xDrive50 Connectivity: Connected Package Pr...
5,iX xDrive50 Audio System: SiriusXM® with 360L ...
6,iX xDrive50 Instrumentation and Controls: Mult...
7,iX xDrive50 Comfort and Convenience: Parking A...
8,iX xDrive50 Safety and Security: Mobility kitA...
9,iX xDrive50 Warranty: BMW Ultimate Care (for c...


## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [475]:
import pandas as pd

df = pd.read_csv(path + 'bmw_webcontent.csv', index_col=['index'])
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 0 to 50
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    51 non-null     object
dtypes: object(1)
memory usage: 816.0+ bytes


In [477]:
df

,text
index,
0,iX xDrive50 Performance and Efficiency: Dual a...
1,"iX xDrive50 Handling, Ride and Braking: xDrive..."
2,iX xDrive50 Exterior: Aluminum Spaceframe with...
3,iX xDrive50 Interior Trim: Perforated Sensatec...
4,iX xDrive50 Connectivity: Connected Package Pr...
5,iX xDrive50 Audio System: SiriusXM® with 360L ...
6,iX xDrive50 Instrumentation and Controls: Mult...
7,iX xDrive50 Comfort and Convenience: Parking A...
8,iX xDrive50 Safety and Security: Mobility kitA...


In [478]:
#Check for duplicates
df.drop_duplicates(['text'], keep='first', inplace=True)

In [479]:
df

,text
index,
0,iX xDrive50 Performance and Efficiency: Dual a...
1,"iX xDrive50 Handling, Ride and Braking: xDrive..."
2,iX xDrive50 Exterior: Aluminum Spaceframe with...
3,iX xDrive50 Interior Trim: Perforated Sensatec...
4,iX xDrive50 Connectivity: Connected Package Pr...
5,iX xDrive50 Audio System: SiriusXM® with 360L ...
6,iX xDrive50 Instrumentation and Controls: Mult...
7,iX xDrive50 Comfort and Convenience: Parking A...
8,iX xDrive50 Safety and Security: Mobility kitA...


In [480]:
#Replace \n in all data for the openai api
df = df.replace('\n', ' ', regex=True)
df = df.replace('(for complete details, visit click here)', ' ', regex=True)
df = df.replace(r'\(\)', ' ', regex=True)
df = df.replace(r'\( \)', ' ', regex=True)


In [481]:
#Set new index
df = df.reset_index(drop=True)


In [482]:
#Convert the content into rag vectors via Open API
#Tokenizer for gpt-3
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i:i+batch_size]["text"].tolist(),
        engine=EMBEDDING_MODEL_NAME
    )
    
    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df

,text,embeddings
0,iX xDrive50 Performance and Efficiency: Dual a...,"[0.003529664594680071, 0.0008285185904242098, ..."
1,"iX xDrive50 Handling, Ride and Braking: xDrive...","[-0.010748385451734066, 0.006585696712136269, ..."
2,iX xDrive50 Exterior: Aluminum Spaceframe with...,"[0.016730420291423798, -0.0007292041555047035,..."
3,iX xDrive50 Interior Trim: Perforated Sensatec...,"[-0.009945431724190712, 0.0020354895386844873,..."
4,iX xDrive50 Connectivity: Connected Package Pr...,"[-0.00783625990152359, -0.0011987572070211172,..."
5,iX xDrive50 Audio System: SiriusXM® with 360L ...,"[0.0017285030335187912, -0.0038443186786025763..."
6,iX xDrive50 Instrumentation and Controls: Mult...,"[-0.019040042534470558, -0.015499262139201164,..."
7,iX xDrive50 Comfort and Convenience: Parking A...,"[0.003894578665494919, 0.0005307366955094039, ..."
8,iX xDrive50 Safety and Security: Mobility kitA...,"[-0.001825525425374508, -0.009621194563806057,..."
9,iX xDrive50 Warranty: BMW Ultimate Care 4-yea...,"[0.006260425318032503, -0.016591470688581467, ..."


In [483]:
df.to_csv(path + "embeddings.csv")

In [ ]:
#For a fresh start
# import numpy as np
# import pandas as pd
# import openai
# openai.api_key = os.getenv('OPENAI_API_KEY')
# df = pd.read_csv("embeddings.csv", index_col=0)
# df["embeddings"] = df["embeddings"].apply(eval).apply(np.array)

## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [484]:
from openai.embeddings_utils import get_embedding, distances_from_embeddings

def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """
    
    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)
    
    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings,
        df_copy["embeddings"].values,
        distance_metric="cosine"
    )
    
    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [485]:
get_rows_sorted_by_relevance("total power output of the M60", df)

,text,embeddings,distances
20,i5 M60 All-Electric Sedan Performance and Effi...,"[-0.0051428284496068954, 0.01057688519358635, ...",0.182865
10,i7 M70 Sedan Performance and Efficiency: Dual ...,"[3.234978794353083e-05, 0.008628277108073235, ...",0.209125
22,"i5 M60 All-Electric Sedan Exterior: 19"" M Aero...","[-0.0065183620899915695, 0.021405735984444618,...",0.214429
21,"i5 M60 All-Electric Sedan Handling, Ride and B...","[-0.010688798502087593, 0.02272859588265419, 0...",0.217926
0,iX xDrive50 Performance and Efficiency: Dual a...,"[0.003529664594680071, 0.0008285185904242098, ...",0.224933
27,i5 M60 All-Electric Sedan Comfort and Convenie...,"[0.01158122718334198, -0.0002572140365373343, ...",0.225096
26,i5 M60 All-Electric Sedan Instrumentation and ...,"[-0.011134861037135124, -0.004490182269364595,...",0.227119
25,i5 M60 All-Electric Sedan Audio System: Sirius...,"[-0.014091280288994312, 0.008873002603650093, ...",0.233364
23,i5 M60 All-Electric Sedan Interior Trim: Anthr...,"[-0.014083806425333023, 0.008574474602937698, ...",0.233466
12,"i7 M70 Sedan Exterior: 21"" M Aerodynamic bi-co...","[-0.009694165550172329, 0.013924463652074337, ...",0.240355


In [511]:
#Prompt builder 
import tiktoken

def create_prompt(question : str, df : pd.DataFrame, max_token_count : int):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, say "I don't know"

Context: 

{}

---

Question: {}
Answer:"""
    
    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

In [487]:
create_prompt('What is the total power output of the M60?', df=df, max_token_count=200 )

'\nAnswer the question based on the context below, and if the question\ncan\'t be answered based on the context, say "I don\'t know"\n\nContext: \n\ni5 M60 All-Electric Sedan Performance and Efficiency: Dual all-electric motors with a total power output of 593 hp and 586 lb ft of torqueBrake Energy Recuperation system84.3 kWh high-voltage lithium-ion battery pack with advanced active thermal management systemFlexible Fast Charging Cable with 110V/220V capabilityJ1772 AC Fast charging capability (11 kW)DC Fast charging capability (SAE Combo connector)\n\n---\n\nQuestion: What is the total power output of the M60?\nAnswer:'

In [449]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150, basic : bool = False
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    if not basic:
        prompt = create_prompt(question, df, max_prompt_tokens)
    else:
        prompt = question
        
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [488]:
#Cut and paste
#Here it is interesting to seee which M60 the rag and the llm prefers
question_one : str =  'What is the total power output of the M60?' 

In [489]:
answer_question(question_one, df)

'The total power output of the M60 is 593 hp.'

In [ ]:
#right answer

In [450]:
#Now let's do the vanilla test
answer_question(question_one, df, basic=True)

'The M60 machine gun has a maximum rate of fire of 550-650 rounds per minute. At a rate of 600 rounds per minute, the total power output of the M60 would be 18,000 rounds per hour. This would equate to approximately 1,800 kilowatts per hour (or 1.8 megawatts) of power output. However, as a firearm, the M60 does not have a measurable power output like an engine or motor.'

In [490]:
#Interestin answer it refers to a machine gun. Second try and give some more context in the prompt
question_one_extend : str =  'What is the total power output of the i5 M60 from BMW?' 
answer_question(question_one_extend, df, basic=True)

'Unfortunately, there is no information available about an i5 M60 from BMW. Can you please provide more context or details?'

### Question 2

In [506]:
#warranty : 4-year/50,000-mile New Vehicle Limited Warranty for Passenger Cars and Light Trucks 2024 Models (valid only in the USA including Puerto Rico)
question_two : str = "What and where covers the BMW Ultimate Care ?"
answer_question(question_two, df)

'The BMW Ultimate Care covers a 4-year/50,000-mile New Vehicle Limited Warranty for Passenger Cars and Light Trucks for the 2024 models, valid in the USA including Puerto Rico.'

In [504]:
#Now let's do the vanilla test
answer_question(question_two, df, basic=True)

'BMW Ultimate Care is a service and maintenance plan that covers regular scheduled services and select wear-and-tear repairs for new BMW vehicles. It is included with the purchase or lease of a new BMW and is valid for the first 3 years or 36,000 miles, whichever comes first. The plan covers services such as engine oil and filter changes, brake fluid flush, wiper blade replacements, and more. It can be utilized at any authorized BMW dealership in the United States.'

In [ ]:
#a little outdated information and seems it was trainied on us sites otherwise there should be a lot of options worldwide

## Extension

In [539]:
def create_prompt2(question : str, df : pd.DataFrame, max_token_count : int, max_answer_tokens : int , history  : str = None,):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")
    
    # Count the number of tokens in the prompt template and question
    prompt_template = """
Answer the question based on the context below, and if the question
can't be answered based on the context, or ask for more details"

Context: 

{}

---

Question: {}
Answer:"""
    
    con_prompt_template = """
Context: 

{}

---

Question: {}
Answer:"""
    #if there is a history and the history togehter with the question is smaller the max count we 
    # choice the continues template otherwise a new question
    if (history != None) and (len(tokenizer.encode(history)) + \
            len(tokenizer.encode(question)) < max_token_count):
            prompt_template = con_prompt_template
            prompt_template = history + prompt_template 

            

    current_token_count = len(tokenizer.encode(prompt_template)) + \
                            len(tokenizer.encode(question))
    
    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:
        
        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count
        
        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

In [533]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"

def answer_question_2(
    question, df, max_prompt_tokens=1800, max_answer_tokens=150, history : str = None
):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model
    
    If the model produces an error, return an empty string
    """
    if history == None:
        prompt = create_prompt2(question, df, max_prompt_tokens)
    else:
        #calculate the whole amount of tokens
        prompt = create_prompt2(question, df, max_prompt_tokens, max_answer_tokens, history)
    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME,
            prompt=prompt,
            max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip(), prompt
    except Exception as e:
        print(e)
        return ""

In [537]:
eof : bool = False
history : str = ''
while(True):
    question = input(prompt='Please enter a text')
    if len(question) == 0: 
        break
    answer = answer_question_2(question=question,df=df, history=history)
    if len(answer)==0:
        history=''
    else:
        hist = history + answer[0]
    print(question+'\n')
    print(answer[0] + '\n')

Tell me some feature of an m60

Some features of an M60 include:

1. Variable Sport Steering: This feature allows for more precise and responsive handling, giving the driver better control over the vehicle.

2. Adaptive Suspension: The M60 comes with an electronically controlled damper that adjusts the suspension based on road conditions, providing a smoother and more comfortable ride for passengers.

3. M Sport Brakes: These brakes, with blue calipers, offer excellent stopping power and improved braking performance.

4. Dynamic Stability Control (DSC): This technology helps prevent the vehicle from losing control in varying driving situations, providing added safety and stability.

5. 4-wheel disc brakes with Anti-lock Braking System (ABS): The ABS helps prevent the brakes from locking up during emergency braking situations,

and the horse power?

The i7 M70 Sedan has 650hp and the iX xDrive50 has 516-hp. The i5 M60 All-Electric has 593 hp.

